# Optimizer 튜토리얼: AdamW와 Muon 비교

이 노트북은 `models.default_model`의 소형 Transformer를 사용하여 옵티마이저 AdamW와 Muon의 학습 경향을 비교합니다. 학습 데이터는 `resource/fineweb_llama32_128k_tokens` 경로에 저장된 FineWeb 10BT 스트리밍 샘플을 메타 Llama 3.2 토크나이저로 전처리한 12.8만 토큰입니다.

## 실험 개요

- 기본 모델: `TransformerForCausalLM` (eager self-attention, 2층, hidden size 128)
- 데이터: FineWeb 10BT 스트리밍 샘플에서 추출한 128k 토큰 (`resource/fineweb_llama32_128k_tokens`)
- 비교 옵티마이저: `torch.optim.AdamW` vs `models.example_optimizer.Muon`
- 평가 지표: 학습 스텝마다 기록한 크로스엔트로피 손실

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from pathlib import Path

from datasets import load_from_disk
from transformers import AutoTokenizer

from models.default_model import TransformerForCausalLM, TransformerConfig
from models.example_optimizer import Muon

TOKEN_DATA_PATH = Path("resource/fineweb_llama32_128k_tokens")

torch.manual_seed(0)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
if tokenizer.pad_token is None and tokenizer.eos_token is not None:
    tokenizer.pad_token = tokenizer.eos_token

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 디바이스: {device}")
print(f"토큰 데이터 경로: {TOKEN_DATA_PATH.resolve()}")

## FineWeb 토큰 데이터 준비

미리 스트리밍 방식으로 수집해둔 `resource/fineweb_llama32_128k_tokens` Arrow 데이터셋에는 Llama 3.2 토크나이저로 인코딩한 토큰 ID가 저장되어 있습니다. 이를 불러와 일정 길이의 시퀀스로 잘라 PyTorch 데이터셋을 구성합니다.

In [ ]:
class TokenChunkDataset(Dataset):
    def __init__(self, tokens: torch.Tensor, seq_len: int, vocab_size: int):
        if tokens.ndim != 1:
            tokens = tokens.view(-1)
        usable = (tokens.numel() // seq_len) * seq_len
        if usable == 0:
            raise ValueError("토큰 수가 시퀀스 길이보다 작습니다.")

        self.tokens = tokens[:usable]
        self.seq_len = seq_len
        self.vocab_size = vocab_size
        self.num_sequences = usable // seq_len

    def __len__(self) -> int:
        return self.num_sequences

    def __getitem__(self, idx: int) -> torch.Tensor:
        start = idx * self.seq_len
        end = start + self.seq_len
        return self.tokens[start:end]

In [ ]:
arrow_dataset = load_from_disk(str(TOKEN_DATA_PATH))
flat_tokens = arrow_dataset[0]["input_ids"]
token_tensor = torch.tensor(flat_tokens, dtype=torch.long)

SEQ_LEN = 128
BATCH_SIZE = 16

dataset = TokenChunkDataset(token_tensor, seq_len=SEQ_LEN, vocab_size=tokenizer.vocab_size)

print(
    f"총 토큰 수: {token_tensor.numel():,}, 사용 가능한 토큰: {len(dataset) * SEQ_LEN:,}, "
    f"시퀀스 수: {len(dataset)}, 시퀀스 길이: {SEQ_LEN}"
)
print(f"배치 크기: {BATCH_SIZE}, vocab_size: {dataset.vocab_size}")

## 기본 Transformer 구성

`TransformerConfig`를 직접 생성해 작은 모델을 정의합니다. attention 구현은 기본값인 eager를 그대로 사용하고, 컨텍스트 길이는 128 토큰으로 제한합니다.

In [ ]:
def build_small_config(vocab_size: int) -> TransformerConfig:
    return TransformerConfig(
        base_model_name_or_path=None,
        vocab_size=vocab_size,
        hidden_size=128,
        intermediate_size=256,
        num_hidden_layers=2,
        num_attention_heads=4,
        num_key_value_heads=2,
        max_position_embeddings=128,
        attention_dropout=0.0,
    )

## 공통 학습 루프 정의

두 옵티마이저가 동일한 초기 가중치에서 시작하도록 학습 전에 난수 시드를 고정합니다. 각 스텝마다 손실을 기록해 이후 그래프로 비교합니다.

In [ ]:
def train_with_optimizer(optimizer_name: str, steps: int = 200) -> torch.Tensor:
    torch.manual_seed(1234)

    config = build_small_config(vocab_size=dataset.vocab_size)
    model = TransformerForCausalLM(config).to(device)
    model.train()

    if optimizer_name.lower() == "adamw":
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=2e-3,
            betas=(0.9, 0.95),
            weight_decay=0.01,
        )
    elif optimizer_name.lower() == "muon":
        optimizer = Muon(
            model.parameters(),
            lr=2e-2,
            momentum=0.95,
            weight_decay=0.01,
        )
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    def create_loader() -> DataLoader:
        return DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True,
            drop_last=True,
            generator=torch.Generator().manual_seed(2024),
        )

    data_loader = create_loader()
    data_iter = iter(data_loader)

    losses = []

    for step in range(1, steps + 1):
        try:
            batch = next(data_iter)
        except StopIteration:
            data_loader = create_loader()
            data_iter = iter(data_loader)
            batch = next(data_iter)

        batch = batch.to(device)
        labels = batch.clone()

        optimizer.zero_grad()
        outputs = model(input_ids=batch, labels=labels)
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        losses.append(loss.item())

    return torch.tensor(losses, dtype=torch.float32)

## 실험 실행

In [ ]:
total_steps = 200

adamw_losses = train_with_optimizer("adamw", steps=total_steps)
muon_losses = train_with_optimizer("muon", steps=total_steps)

comparison = {
    "AdamW": adamw_losses,
    "Muon": muon_losses,
}

for name, losses in comparison.items():
    print(f"{name:5s} | 초기 loss: {losses[0]:.3f} -> 최종 loss: {losses[-1]:.3f}")

## 결과 시각화

손실 곡선은 5 스텝 이동 평균으로 부드럽게 만들어 비교합니다.

In [ ]:
def moving_average(values: torch.Tensor, window: int = 5) -> np.ndarray:
    array = values.cpu().numpy()
    if len(array) < window:
        return array
    kernel = np.ones(window, dtype=np.float32) / window
    return np.convolve(array, kernel, mode="valid")

plt.figure(figsize=(8, 4))

for name, losses in comparison.items():
    smoothed = moving_average(losses, window=5)
    plt.plot(
        np.arange(len(smoothed)),
        smoothed,
        label=name,
    )

plt.xlabel("Training step")
plt.ylabel("Loss (moving average)")
plt.title("AdamW vs Muon 학습 곡선")
plt.legend()
plt.grid(True, alpha=0.2)
plt.show()

## 마무리

- 이동 평균 손실 곡선을 통해 두 옵티마이저의 수렴 속도와 안정성을 직관적으로 비교할 수 있습니다.
- Muon은 직교화 기반 업데이트를 사용하므로 행렬 파라미터가 많은 레이어에서 더 공격적인 학습률을 사용해도 안정적인 경향을 보입니다.
- AdamW는 더 익숙한 기본 옵티마이저이므로, 기준선으로 활용하고 추가 실험에서 하이퍼파라미터를 조정해볼 수 있습니다.

필요에 따라 `train_with_optimizer`의 학습 스텝 수나 하이퍼파라미터를 조정하며 다양한 시나리오를 실험해보세요.